In [28]:
using Printf
using CPUTime
include("model_POMDP.jl")
include("parser.jl")
include("spu_POMDP.jl")
using Gurobi

function createFolder(directory)
    try
        if isdir(directory) == false
            mkdir(directory)
        end
    catch
        println(string("Error:Creating directory ", directory))
    end
end

createFolder (generic function with 1 method)

# Draw randomly the parameters of the influence diagram

In [29]:
@printf "================================ Draw parameters =========================================\n"
T = 5
S = 1:4
O = 1:4
A = 1:4

#p_init(s) = P(S_1=s)
p_init = rand(length(S))
p_init = p_init ./ sum(p_init)

#p_trans(s',a,s) = P(S_{t+1} = s' |  S_t = s, A_t=s)
p_trans = rand(length(S),length(A),length(S))
p_trans = p_trans ./ sum(p_trans,dims=3)

# # p_o(o,s) = P(O_t = o | S_t =s)
p_emis = rand(length(S),length(O))
p_emis = p_emis ./ sum(p_emis,dims=2)

#reward(s,a,s') = r(S_t=s, A_t=a, S_{t+1}=s')
reward = 10*rand(length(S),length(A), length(S))
@printf "================================ Done =========================================\n"

================================ Draw parameters =========================================
================================ Done =========================================


# Preprocessing of cut probabilities

In [30]:
@printf "================== Compute probabilities for valid cuts =======================\n"
#p(s_{t}|s_{t-1}a_{t-1}o_t)
p_cuts = zeros(length(S),length(A),length(S), length(O))

for s in S, a in A, ss in S, oo in O
    if sum(p_trans[s,a,sss]*p_emis[sss,oo] for sss in S) == 0
	p_cuts[s,a,ss,oo] = 0
    else
    	p_cuts[s,a,ss,oo] = (p_trans[s,a,ss]*p_emis[ss,oo])/(sum(p_trans[s,a,sss]*p_emis[sss,oo] for sss in S))
    end
end
@printf "================================ Done =========================================\n"

================== Compute probabilities for valid cuts =======================
================================ Done =========================================


# Run SPU algorithm

In [31]:
#############################SPU algorithm##################################
@printf "================================ SPU algorithm =========================================\n"
@time SPU_Time = SPU_obj, SPU_delta, iterations = SPU_algorithm(T,S,O,A,p_init,p_emis,p_trans,reward)
SPU_delta,μ_s,μ_sa,μ_soa,μ_sasoa = potential_warmstart_cuts(T,S,O,A,p_init,p_emis,p_trans,SPU_delta)
@printf "================================ Done =========================================\n"

================================ SPU algorithm =========================================
=============Cycle 0==================
=============Cycle 1==================
  1.246745 seconds (2.12 M allocations: 100.793 MiB, 3.26% gc time)
================================ Done =========================================


# Solve MILP on Polytope $\overline{\mathcal{Q}}^1$

In [32]:
#############################Polytope Q^1####################################
@printf "============================ Solve the MILP and its relaxation ===============================\n"
#Define the bounds b in McCormick inequalities
#b=1
bounds = ones(T,length(S),length(O))

#Solve the relaxation
m = model(T,S,O,A,p_init,p_trans,p_emis,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_p1_relax = objective_value(m_lp)

#Solve the MILP
m = model(T,S,O,A,p_init,p_trans,p_emis,reward,bounds)
m = set_initial_solution(m,T,S,O,A,SPU_delta,μ_s,μ_sa,μ_soa,μ_sasoa)
optimize!(m)

#Set the results
milp_time_p1 = solve_time(m)
milp_obj_p1 = objective_value(m)
milp_bestbound_p1 = objective_bound(m)
milp_gap_p1 = ((milp_bestbound_p1 - milp_obj_p1)/milp_bestbound_p1)*100
gap_p1_with_spu = ((milp_obj_p1 - SPU_obj)/SPU_obj)*100
@printf "============================ Done ===============================\n"

============================ Solve the MILP and its relaxation ===============================
Academic license - for non-commercial use only
Academic license - for non-commercial use only
============================ Done ===============================


# Solve MILP on Polytope $\overline{\mathcal{Q}}^b$

In [33]:
#################################Polytope Q^b##########################
@printf "============================ Solve the MILP and its relaxation ===============================\n"
#Define the bounds b in McCormick inequalities
#Compute the bounds using propagation
bounds = compute_bounds(T,S,O,A,p_init,p_trans,p_emis,reward)

#Solve the relaxation
m = model(T,S,O,A,p_init,p_trans,p_emis,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_pb_relax = objective_value(m_lp)

#Solve the MILP
m = model(T,S,O,A,p_init,p_trans,p_emis,reward,bounds)
m = set_initial_solution(m,T,S,O,A,SPU_delta,μ_s,μ_sa,μ_soa,μ_sasoa)
optimize!(m)

#Set the results
milp_time_pb = solve_time(m)
milp_obj_pb = objective_value(m)
milp_bestbound_pb = objective_bound(m)
milp_gap_pb = ((milp_bestbound_pb - milp_obj_pb)/milp_bestbound_pb)*100
gap_pb_with_spu = ((milp_obj_pb - SPU_obj)/SPU_obj)*100
@printf "============================ Done ===============================\n"

============================ Solve the MILP and its relaxation ===============================
Academic license - for non-commercial use only
Academic license - for non-commercial use only
============================ Done ===============================


# Solve MILP on Polytope $\mathcal{Q}^{\perp \!\!\! \perp,1}$

In [34]:
#################################Polytope Q^free##########################
@printf "============================ Solve the MILP and its relaxation ===============================\n"
#Define the bounds b in McCormick inequalities
#b=1
bounds = ones(T,length(S),length(O))

#Solve the linear relaxation
m = model_cuts(T,S,O,A,p_init,p_trans,p_emis,p_cuts,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_pfree_relax = objective_value(m_lp)

#Solve the MILP
m = model_cuts(T,S,O,A,p_init,p_trans,p_emis,p_cuts,reward,bounds)
m = set_initial_solution_cuts(m,T,S,O,A,SPU_delta,μ_s,μ_sa,μ_soa,μ_sasoa)
optimize!(m)

#Set the results
milp_time_pfree = solve_time(m)
milp_obj_pfree = objective_value(m)
milp_bestbound_pfree = objective_bound(m)
milp_gap_pfree = ((milp_bestbound_pfree-milp_obj_pfree)/milp_bestbound_pfree)*100
gap_pfree_with_spu = ((milp_obj_pfree - SPU_obj)/SPU_obj)*100
@printf "============================ Done ===============================\n"

============================ Solve the MILP and its relaxation ===============================
Academic license - for non-commercial use only
Academic license - for non-commercial use only
============================ Done ===============================


# Solve MILP on Polytope $\mathcal{Q}^{\perp \!\!\! \perp,b}$

In [35]:
#################################Polytope Q^free \cap P^b##########################
@printf "===================== Solve the MILP and its relaxation ===============================\n"
#Define the bounds b in McCormick inequalities
#Compute the bounds using propagation
bounds = compute_bounds(T,S,O,A,p_init,p_trans,p_emis,reward)

#Solve the linear relaxation
m = model_cuts(T,S,O,A,p_init,p_trans,p_emis,p_cuts,reward,bounds)
m_lp = relax_model(m)
optimize!(m_lp)
milp_obj_pfree_pb_relax = objective_value(m_lp)

#Solve the MILP
m = model_cuts(T,S,O,A,p_init,p_trans,p_emis,p_cuts,reward,bounds)
m = set_initial_solution_cuts(m,T,S,O,A,SPU_delta,μ_s,μ_sa,μ_soa,μ_sasoa)
optimize!(m)

#Set the results
milp_time_pfree_pb = solve_time(m)
milp_obj_pfree_pb = objective_value(m)
milp_bestbound_pfree_pb = objective_bound(m)
milp_gap_pfree_pb = ((milp_bestbound_pfree_pb-milp_obj_pfree_pb)/milp_bestbound_pfree_pb)*100
gap_pfree_pb_with_spu = ((milp_obj_pfree_pb - SPU_obj)/SPU_obj)*100
@printf "============================ Done ===============================\n"

===================== Solve the MILP and its relaxation ===============================
Academic license - for non-commercial use only
Academic license - for non-commercial use only
============================ Done ===============================


In [36]:
#########################################Results##############################
@printf "===========================================================\n"
@printf "==============================================\n"
@printf "=============Results %.0f %.0f %.0f %.0f ==================\n" T length(S) length(O) length(A)

@printf "Q^1 Integrality gap : %f \n" (1-milp_obj_p1/milp_obj_p1_relax)*100
@printf "Q^1 Time : %f \n" milp_time_p1
@printf "Q^1 Final gap value: %f \n" milp_gap_p1
@printf "Q^1 Gap with SPU: %f \n \n" gap_p1_with_spu

@printf "Q^b Integrality gap : %f \n" (1- milp_obj_pb/milp_obj_pb_relax)*100
@printf "Q^b Time : %f \n" milp_time_pb
@printf "Q^b Final gap value: %f \n" milp_gap_pb
@printf "Q^b Gap with SPU: %f \n \n" gap_pb_with_spu


@printf "Q^free Integrality gap : %f \n" (1- milp_obj_pfree/milp_obj_pfree_relax)*100
@printf "Q^free Time : %f \n" milp_time_pfree
@printf "Q^free Final gap value: %f \n" milp_gap_pfree
@printf "Q^free Gap with SPU: %f \n \n" gap_pfree_with_spu


@printf "Q^{free,b} Integrality gap : %f \n" (1- milp_obj_pfree_pb/milp_obj_pfree_pb_relax)*100
@printf "Q^{free,b} Time : %f \n" milp_time_pfree_pb
@printf "Q^{free,b} Final gap value: %f \n" milp_gap_pfree_pb
@printf "Q^{free,b} Gap with SPU: %f \n \n" gap_pfree_pb_with_spu

=============Results 5 4 4 4 ==================
Q^1 Integrality gap : 18.703455 
Q^1 Time : 7.233640 
Q^1 Final gap value: 0.000000 
Q^1 Gap with SPU: 0.000003 
 
Q^b Integrality gap : 9.319626 
Q^b Time : 3.229853 
Q^b Final gap value: 0.000000 
Q^b Gap with SPU: 0.000013 
 
Q^free Integrality gap : 8.544173 
Q^free Time : 3.705839 
Q^free Final gap value: 0.000000 
Q^free Gap with SPU: 0.000000 
 
Q^{free,b} Integrality gap : 2.817550 
Q^{free,b} Time : 2.002128 
Q^{free,b} Final gap value: 0.000000 
Q^{free,b} Gap with SPU: 0.000000 
 
